# Creating a Basic IP Packet

In [1]:
from scapy.all import *

In [2]:
# Create an IP packet with a source and destination IP address
ip_packet = IP(src="192.168.1.1", dst="192.168.1.2")

In [3]:
# Add a TCP layer with source and destination ports
tcp_packet = ip_packet / TCP(sport=12345, dport=80)

In [4]:
# Display the packet
tcp_packet.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.1.1
  dst       = 192.168.1.2
  \options   \
###[ TCP ]###
     sport     = 12345
     dport     = http
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []



# Adding Payload and Custom Data

In [5]:
# Adding a payload with custom data
packet_with_data = tcp_packet / Raw(load="GET / HTTP/1.1\r\nHost: example.com\r\n\r\n")

In [6]:
# Display the packet with payload
packet_with_data.show()

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 192.168.1.1
  dst       = 192.168.1.2
  \options   \
###[ TCP ]###
     sport     = 12345
     dport     = http
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = []
###[ Raw ]###
        load      = b'GET / HTTP/1.1\r\nHost: example.com\r\n\r\n'



# Sending a Packet

In [7]:
# Send the created packet
send(packet_with_data)

.
Sent 1 packets.


# Sniffing Packets

The sniff() function is one of Scapy's core packet capture functions that allows you to intercept network traffic. If we use `sniff(timeout=10, prn=lambda x: x.show())` as per the lab instructions:

+ timeout=10: This sets the packet capturing to run for exactly 10 seconds before stopping automatically.   
+ prn=lambda x: x.show(): This is a callback function that determines what to do with each captured packet.   
    + prn stands for "print" (though it's more general than that)
    + The lambda x: x.show() is a small anonymous function that takes each packet (x) and calls the show() method on it
    + The show() method displays a detailed breakdown of the packet structure, showing all layers (Ethernet, IP, TCP, etc.)

The output is the detailed structure of packets that the container captured during those 10 seconds. Each packet shows multiple network layers, from the lowest (Ethernet) to the highest (application data).

From the output, I've captured HTTP traffic between my host machine and the Docker container on port 8888, which is the Jupyter server. The packet contains HTTP headers including authentication tokens and cookies.

**Important security note: The captured packet contains sensitive information including:**

+ An authentication token for the Jupyter server
+ A CSRF token
+ Cookie values

This demonstrates one reason why network sniffing tools require special privileges - they can capture sensitive information traveling over the network. You should be careful about sharing the complete output, as these tokens could potentially allow someone to access your Jupyter server if it's accessible from the internet.

This is also a great example of why encrypted protocols like HTTPS are important - they would prevent this kind of sensitive information from being visible in a packet capture.
Since this is just a temporary learning server running in a local Docker container, the security risk is minimal. Let me explain why:

The Jupyter server is only accessible from my local machine (127.0.0.1 or localhost), which means only someone with access to my computer could potentially use these tokens.
Docker containers typically have internal networking that isn't directly exposed to the internet unless specifically configured that way.
The tokens shared have a limited lifespan - Jupyter tokens are typically valid only for the current session and expire when the server restarts.
This is all within an educational context where I am specifically learning about packet sniffing and network analysis.

In any case, I am sanitising the payload when pushing to git:

In [15]:
from scapy.all import sniff, IP, TCP, Raw
import re

# Function to sanitize potentially sensitive information
def sanitize_packet(packet):
    """
    Create a sanitized version of a packet by hiding sensitive information
    like authentication tokens, cookies, passwords, etc.
    """
    # Make a copy of the packet to avoid modifying the original
    sanitized = packet.copy()
    
    # Check if the packet has raw payload data (like HTTP headers)
    if Raw in sanitized:
        # Try to decode the payload as text
        try:
            payload = sanitized[Raw].load
            payload_text = payload.decode('utf-8', errors='replace')
            
            # Define patterns to sanitize
            patterns = [
                # Authentication tokens
                (r'Authorization: [^\r\n]+', 'Authorization: [TOKEN REDACTED]'),
                # Cookies
                (r'Cookie: [^\r\n]+', 'Cookie: [COOKIES REDACTED]'),
                # CSRF/XSRF tokens
                (r'X-CSRF-Token: [^\r\n]+', 'X-CSRF-Token: [CSRF TOKEN REDACTED]'),
                (r'X-XSRFToken: [^\r\n]+', 'X-XSRFToken: [XSRF TOKEN REDACTED]'),
                # JWT tokens (look like: eyJ...)
                (r'eyJ[a-zA-Z0-9_-]+\.[a-zA-Z0-9_-]+\.[a-zA-Z0-9_-]+', '[JWT TOKEN REDACTED]'),
                # Basic auth credentials
                (r'Basic [a-zA-Z0-9+/=]+', 'Basic [CREDENTIALS REDACTED]'),
                # Email addresses
                (r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[EMAIL REDACTED]')
            ]
            
            # Apply all sanitization patterns
            for pattern, replacement in patterns:
                payload_text = re.sub(pattern, replacement, payload_text, flags=re.IGNORECASE)
            
            # Update the packet with sanitized payload
            sanitized[Raw].load = payload_text.encode('utf-8')
        except:
            # If we can't decode the payload as text, just mark it as binary data
            sanitized[Raw].load = b'[BINARY DATA REDACTED]'
    
    return sanitized

# Function to display packets with sensitive info removed
def safe_packet_display(packet):
    """Display a packet after sanitizing sensitive information"""
    # Sanitize the packet
    safe_packet = sanitize_packet(packet)
    
    # Display the sanitized packet
    safe_packet.show()
    
    # Return None to prevent storing packets in memory
    return None

# Main function to capture packets
def capture_packets(count=5, timeout=10, filter_str=None):
    """
    Capture network packets and display them with sensitive information hidden.
    
    Parameters:
        count: Maximum number of packets to capture
        timeout: Maximum time to capture packets (in seconds)
        filter_str: BPF filter string (e.g., "tcp port 80" for HTTP traffic)
    """
    print(f"Starting packet capture:")
    print(f"- Maximum packets: {count}")
    print(f"- Timeout: {timeout} seconds")
    if filter_str:
        print(f"- Filter: {filter_str}")
    
    print("\nCapturing packets... Press Ctrl+C to stop early.")
    
    # Start sniffing with our safe display function
    sniff(count=count, timeout=timeout, prn=safe_packet_display, filter=filter_str, store=False)
    
    print("\nPacket capture complete!")

# Example usage - capture 5 packets or run for 10 seconds
# For HTTP traffic only, you could use: capture_packets(filter_str="tcp port 80")
# For Jupyter traffic only, you could use: capture_packets(filter_str="tcp port 8888")
capture_packets(count=5)

Starting packet capture:
- Maximum packets: 5
- Timeout: 10 seconds

Capturing packets... Press Ctrl+C to stop early.
###[ Ethernet ]###
  dst       = 02:42:67:73:71:de
  src       = 02:42:ac:11:00:02
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 682
     id        = 64439
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xe470
     src       = 172.17.0.2
     dst       = 172.17.0.1
     \options   \
###[ TCP ]###
        sport     = 8888
        dport     = 42942
        seq       = 233670585
        ack       = 379658324
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 501
        chksum    = 0x5ac2
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3973596340, 1991707741))]
###[ Raw ]###
           load      = b'\xef\xbf\xbd~\x02r\x06\x00\x00\x00\x00\x00\x00\x008\x00\x00\x00\x00\x00\x00\x00=\x00\x00\x